In [58]:
small = False

M = 5000 if small else 26501365 # is het aantal stappen

In [131]:
with open('input7' + ('A' if small else 'B') + '.txt', 'r') as i:
    data = list(list(line.strip()) for line in i.readlines())

h = len(data)
w = len(data[0])

graph = {}

def enc(y, x):
    return x * h + y
def dec(v):
    return [v % h, v // h]
ends = []
for y in range(h):
    for x in range(w):
        l = []
        if data[y][x] in '.S':
            if data[y][x] == 'S': start = enc(y, x)
            if y >     0 and data[y - 1][x] in '.S': l += [enc(y - 1, x)]
            if y < h - 1 and data[y + 1][x] in '.S': l += [enc(y + 1, x)]
            if x >     0 and data[y][x - 1] in '.S': l += [enc(y, x - 1)]
            if x < w - 1 and data[y][x + 1] in '.S': l += [enc(y, x + 1)]
            graph[enc(y, x)] = l

nodes = set(graph.keys())

# CHATGPT: Give Python code to find the distance to all nodes from a starting node. 
# The input is a dictionary with nodes as keys and list of connected nodes as values. 
import heapq
def dijkstra(graph, start):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    priority_queue = [(0, start)]
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]: continue
        for neighbor in graph[current_node]:
            distance = current_distance + 1
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    return distances



50 / 10 = ongeveer radius van 5 

Doe eerst een kwart, zonder randen. 
Stel het kost K=132 om tot het hoekje te komen. 
Stel het kost S=131 om een stap naar het volgende blokje te doen (=breedte van blokje)
Stel het kost L (<2*S?) om het verste puntje te vinden in het blokje

Vind k integer z.d.d. M >= K + k * S + L, dan is het aantal volledige blokjes gelijk aan het aantal blokjes bereikbaar in k stappen
Dus k = floor((M - K - L) / S). 
Hoeveel paren (x,y) voldoen aan x + y <= k. Dat is niet moeilijk. 
Hoeveel paren (x,y) voldoen aan k < x + y < k + L / S

In [166]:
richting = 0 # lower-right
hoekjes = {
    0: [0, 0], # lower-right
    1: [h - 1, 0], # upper right
    2: [0, w - 1], # lower left
    3: [h - 1, w - 1], # upper left
}
hoekje_omgekeerd = lambda hoekje_nr: {0: 3, 1: 2, 2: 1, 3: 0}[hoekje_nr]
assert(w == h)
S = w
print(S)

131


In [61]:
L = 2 * (S - 1)
for hoekje in hoekjes.values():
    assert(max(filter(lambda x: x != float('inf'), dijkstra(graph, enc(*hoekje)).values())) == L)

In [148]:
import math
k = math.floor((M - K - L) / S)
print((M - K - L) / S)
print(k)
print(((M - K) - k * S) / S)

202297.5038167939
202297
2.4885496183206106


In [149]:
aantal_per_blok_totaal = len(list(1 for row in data for item in row if item in 'S.'))
print(aantal_per_blok_totaal)

14677


In [74]:
import math
count = lambda start, max_step=math.inf: len([key for key, val in dijkstra(graph, start).items() if val <= max_step and ((max_step if max_step != math.inf else 0) - val) % 2 == 0])

7315

In [113]:
odd = lambda x: ((x - 1) // 2) * 2 + 1
even = lambda x: (x // 2) * 2
sum_odd_numbers = lambda max: ((odd(max) // 2) + 1) ** 2
sum_even_numbers = lambda max: even(max) * (even(max) + 2) // 4
sum_numbers = lambda max, sign: sum_odd_numbers(max) if sign == 1 else sum_even_numbers(max)

In [158]:
k

202297

In [170]:
center = count(start, M)
def chunk(K, start, aantallen):
    start_parity = K % 2
    volledig_gevuld = (aantallen[0] * count(enc(*start), L + start_parity) + 
                    aantallen[1] * count(enc(*start), L + (1 - start_parity))) # onafhankelijk van hoekje
    extra1 = aantallen[2] * count(enc(*hoekjes[0]), M - K - (k + 1) * S)
    extra2 = aantallen[3] * count(enc(*hoekjes[0]), M - K - (k + 2) * S)
    return [volledig_gevuld, extra1, extra2]


c = sum([sum(chunk(dijkstra(graph, start)[enc(*hoekjes[hoekje_omgekeerd(hoekje_nr)])] + 2, hoekjes[hoekje_nr], [sum_numbers(k, 0), sum_numbers(k, 1), (k + 1), (k + 2)])) for hoekje_nr in range(4)])
a = sum([sum(chunk(66, p, [k - (k // 2), k // 2, (k + 1), (k + 2)])) for p in [[0, w // 2], [h - 1, w // 2], [h // 2, 0], [h // 2, w - 1]]]) # not assuming K is odd

totaal = center + c + a


In [171]:
totaal

600339047143451